## Introduction

This notebook shows one of possible solutions to decrypt ciphertext with diff=2.

Notebook is based on the assumption that cipher is transpositional: ciphertext's character frequences are not changed, but are appeared in different order that text's characters. Hypotesis: this is route cipher (columnar or something like that), so we can find such patterns as names of heroes (in uppercase) in reshaped ciphertext.

After some exploration, we can detect the mapping rule for decrypt ciphertext and encrypt text.

PS. In other solutions you can see that this is rail fence cipher (a.k.a. zigzag cipher), so you can decrypt it using brute force only on ciphertext with different key size (rail count). This is much simple, but it needs aprior assumption of rail cipher. Current solution based on more general assumption of route cipher, but it needs a mapped pairs of text and ciphertext.

## Loading and mapped pairs of text and ciphertext detection

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
df_train = pd.read_csv('../input/ciphertext-challenge-iii/train.csv')
df_train.shape

In [ ]:
# Encrypt and decrypt of diff=1

ALPHABET = 'abcdefghijklmnopqrstuvwxy'
KEY = 'pyle'


def __encrypt_char(c, alphabet, key, idx):
    key_size, alphabet_size = len(key), len(alphabet)
    if c in alphabet:
        pos = (alphabet.index(c) + alphabet.index(key[idx])) % alphabet_size
    return alphabet[pos], (idx + 1) % key_size


def __decrypt_char(c, alphabet, key, idx):
    key_size, alphabet_size = len(key), len(alphabet)
    if c in alphabet:
        pos = (alphabet_size + alphabet.index(c) - alphabet.index(key[idx])) % alphabet_size
    return alphabet[pos], (idx + 1) % key_size


def encrypt1(text, alphabet=ALPHABET, key=KEY, shift=0, verbose=False):
    text = ''.join(['a']*shift) + text
    key_size, alphabet_size = len(key), len(alphabet)
    alphabet1, alphabet2 = alphabet.lower(), alphabet.upper()
    key1, key2 = key.lower(), key.upper()
    idx = 0
    ciphertext = ''
    for c in text:
        if verbose:
            print(c, end='')
        if c in alphabet1:
            c, idx = __encrypt_char(c, alphabet1, key1, idx)
        elif c in alphabet2:
            c, idx = __encrypt_char(c, alphabet2, key2, idx)
        if verbose:
            print('->', c)
        ciphertext += c
    return ciphertext[shift:]


def decrypt1(ciphertext, alphabet=ALPHABET, key=KEY, verbose=False):
    key_size, alphabet_size = len(key), len(alphabet)
    alphabet1, alphabet2 = alphabet.lower(), alphabet.upper()
    key1, key2 = key.lower(), key.upper()
    idx = 0
    text = ''
    for c in ciphertext:
        if verbose:
            print(c, end='')
        if c in alphabet1:
            c, idx = __decrypt_char(c, alphabet1, key1, idx)
        elif c in alphabet2:
            c, idx = __decrypt_char(c, alphabet2, key2, idx)
        if verbose:
            print('->', c)
        text += c
    return text

In [ ]:
for shift in range(4):
    df_train[f'text{shift}'] = (''.join(['a']*shift) + df_train['text']).apply(lambda x: encrypt1(x)[shift:])

In [ ]:
df_train.head()

In [ ]:
df_test = pd.read_csv('../input/ciphertext-challenge-iii/test.csv')
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_train['text_len'] = df_train['text'].apply(lambda x: len(x))
df_test['ciphertext_len'] = df_test['ciphertext'].apply(lambda x: len(x))

In [ ]:
mask2 = df_test['difficulty'] == 2
df_test = df_test[mask2]
df_test.head()

In [ ]:
df_test['ciphertext_len'].value_counts()

In [ ]:
df_test['ciphertext_len'].sort_values().tail(10)

In [ ]:
ciphertext1_idx = df_test['ciphertext_len'].sort_values().index[-1]

In [ ]:
df_test.loc[ciphertext1_idx, 'ciphertext']

In [ ]:
df_train['text_len'].sort_values().tail(10)

In [ ]:
df_train.loc[df_train['text_len'].sort_values().tail(10).index, 'text']

In [ ]:
text1_idx = df_train['text_len'].sort_values().tail(10).index[-1]
df_train.loc[text1_idx, 'text3']

## Analyze of character frequencies and more pairs detection

In [ ]:
text1 = df_train.loc[text1_idx, 'text3']
s_text1 = pd.Series(list(text1))

In [ ]:
ciphertext1 = df_test.loc[ciphertext1_idx, 'ciphertext']
s_ciphertext1 = pd.Series(list(ciphertext1))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12,6))
s_text1.value_counts(normalize=True).plot(kind='bar', log=True, alpha=0.25, label='train')
s_ciphertext1.value_counts(normalize=True).plot(kind='bar', log=True, color='orange', alpha=0.25, label='test')
plt.legend()

In [ ]:
def stats(s):
    '''Print text statistics for different characters'''
    print('lower:', len([x for x in s if x.islower()]), end=' ')
    print('upper:', len([x for x in s if x.isupper()]), end=' ')
    print('digit:', len([x for x in s if x.isdigit()]), end=' ')
    print('space:', len([x for x in s if x == ' ']), end=' ')
    print(', -', len([x for x in s if x == ',']), end=' ')
    print('. -', len([x for x in s if x == '.']))
    print('GPLBIXTE:', len([x for x in s if x in 'GPLBIXTE']), end=' ')
    print('GPQOMDYW:', len([x for x in s if x in 'GPQOMDYW']), end=' ')
    print('U:', len([x for x in s if x in 'U']))

In [ ]:
stats(text1)

In [ ]:
stats(ciphertext1)

In [ ]:
pd.Series([x for x in text1 if x.isupper()]).value_counts()

In [ ]:
pd.Series([x for x in ciphertext1 if x.isupper()]).value_counts()

In [ ]:
len(text1), len(ciphertext1)

- facts: 
  - `.` (dot) and `,` (comma) - equal counts but different postitions in both text and ciphertext
  - ` ` (space) - only 2 more in ciphertext (by random generation)

In [ ]:
dot_p = [pos for pos, char in enumerate(text1) if char == '.']
dot_c = [pos for pos, char in enumerate(ciphertext1) if char == '.']
dot_U = [pos for pos, char in enumerate(ciphertext1) if char == 'U']

In [ ]:
plt.title('Dots postitions')
plt.step(range(len(dot_p)), dot_p, label='text')
plt.step(range(len(dot_c)), dot_c, label='ciphertext')
plt.legend()

In [ ]:
comma_p = [pos for pos, char in enumerate(text1) if char == ',']
comma_c = [pos for pos, char in enumerate(ciphertext1) if char == ',']

In [ ]:
plt.title('Commas postitions')
plt.step(range(len(comma_p)), comma_p, label='text')
plt.step(range(len(comma_c)), comma_c, label='ciphertext')
plt.legend()

In [ ]:
U_p = [pos for pos, char in enumerate(text1) if char == 'U']
U_c = [pos for pos, char in enumerate(ciphertext1) if char == 'U']

In [ ]:
plt.title('U postitions')
plt.step(range(len(U_p)), U_p, where='post', label='text')
plt.plot(range(len(U_p)), U_p, 'C0o')
plt.step(range(len(U_c)), U_c, where='post', label='ciphertext')
plt.plot(range(len(U_c)), U_c, 'C1o')
plt.legend()

- hypotesis:
  - positions of `.` and `,` are changed
  - transposition cipher (i.e. columnar) nx10
  - key size candidates:
    - 5, 10, 20, 25, 50

In [ ]:
ciphertext2_idx = df_test['ciphertext_len'].sort_values().index[-2]
ciphertext3_idx = df_test['ciphertext_len'].sort_values().index[-3]

In [ ]:
ciphertext2 = df_test.loc[ciphertext2_idx, 'ciphertext']
ciphertext2

In [ ]:
ciphertext3 = df_test.loc[ciphertext3_idx, 'ciphertext']
ciphertext3

In [ ]:
df_train[(df_train['text_len'] > 300) & (df_train['text_len'] <= 400)]

In [ ]:
text2_idx = 14716
text2 = df_train.loc[text2_idx, 'text1']
text2

In [ ]:
text3_idx = 61720
text3 = df_train.loc[text3_idx, 'text0']
text3

In [ ]:
s_text123 = pd.Series([y for x in [text1, text2, text3] for y in list(x)])
s_ciphertext123 = pd.Series([y for x in [ciphertext1, ciphertext2, ciphertext3] for y in list(x)])

In [ ]:
plt.figure(figsize=(12,6))
s_text1.value_counts(normalize=True).plot(kind='bar', log=True, alpha=0.25, label='train')
s_ciphertext1.value_counts(normalize=True).plot(kind='bar', log=True, color='orange', alpha=0.25, label='test')
plt.legend()

In [ ]:
stats(text2)

In [ ]:
stats(ciphertext2)

- text2 and ciphertext2 has same count of ` `, `,` and `.`

In [ ]:
stats(text3)

In [ ]:
stats(ciphertext3)

- text3 and ciphertext3 has same count of `.` and almost the same `,` and ` `
- ciphertext3 has 8 digits, but text3 has 0 digits!
  - all digits in ciphertext3 are from left|right paddings

## Reshaping ciphertext and UPPERCASE patterns detection

In [ ]:
text3

In [ ]:
def size(text, other_side):
    '''Detecting size of rectangle to fit text, using given other side'''
    return (len(text) - 1) // other_side + 1

In [ ]:
def reshape(text, n_rows=None, n_cols=50, pattern=None, wide=True, verbose=False):
    '''Reshape text into rectangle by rows or columns with output'''
    if verbose:
        print(text, '\n')
    if n_rows is not None:
        if verbose:
            row_size = size(text, n_rows)
            for i in range(n_rows):
                row = text[i*row_size:(i+1)*row_size]
                if wide:
                    print(' '.join(list(row)))        
                else:
                    print(row)
        return text
    res = ''
    n_rows = size(text, n_cols)
    for i in range(n_rows):
        row = text[i::n_rows]
        res += row
        if verbose:
            if wide:
                print(' '.join(list(row)))
            else:
                print(row)
    if pattern is not None:
        if pattern in res:
            print(pattern, '->->->', n_cols)
        if pattern[::-1] in res:
            print(pattern, '<-<-<-', n_cols)
    return res

In [ ]:
# np.array(list(ciphertext3)).reshape(20, -1).T

In [ ]:
def find_pattern(cipertext, patterns):
    '''Find given patterns in ciphertext in direct or backward direction for different key size'''
    for key_size in [5, 10, 20, 25, 50, 100]:
        print('----- key_size -----', key_size)
        for pattern in patterns:
            reshape(cipertext, n_cols=key_size, pattern=pattern)

In [ ]:
find_pattern(ciphertext3, ['BEVD', 'RWIDCPV', 'JTMFSD', 'BLMKR'])

In [ ]:
text3

In [ ]:
reshape(ciphertext3, n_cols=20, verbose=True)

In [ ]:
reshape(ciphertext3, n_rows=20, verbose=True)

In [ ]:
text3

- column key_size=20
- there are names uppercase letters 
  - forward direction (`BLMKR`, `JTMFSD`)
  - backward direction (`RWIDCPV`)
- hypotesis: route cipher
- last column not matching

In [ ]:
from PIL import Image
img = Image.open('../input/diff2-mapping/photo5219986312940071919.jpg')
img

In [ ]:
def shift_row(row_idx, text, n_cols, shift):
    row = text[row_idx*n_cols:(row_idx+1)*n_cols]
    return row[shift:] + row[:shift]

In [ ]:
def decrypt(ciphertext, n_cols=20, verbose=False):
    '''First version'''
    #1. extract head and tail rows
    d = n_cols//2
    head, tail = ciphertext[:d], ciphertext[-d:]
    if verbose:
        print('head:', head, 'tail:', tail)
    #2. swap right and left cols
    ciphertext_ = ''
    n_rows = size(ciphertext, n_cols)
    for i in range(n_rows):
        ciphertext_ += shift_row(i, ciphertext, n_cols, d)
    ciphertext = ciphertext_
    #3. extracting cols
    cols = []
    for i in range(n_cols):
        col = ciphertext[i::n_cols]
        if i < d:
            col = col[:-1]  
        else:
            col = col[1:]
        if i % 2 == 0:
            col = head[i//2] + col
        else:
            col = tail[i//2] + col[::-1]
        if verbose:
            print('i:', i, 'col:', col)
        cols += col
    return ''.join(cols)

In [ ]:
text3d = decrypt(ciphertext3, verbose=True)

In [ ]:
text3

In [ ]:
text3d

In [ ]:
text3 in text3d

In [ ]:
ciphertext3[:10], ciphertext3[-10:]

In [ ]:
reshape(ciphertext3[10:-10], n_rows=19, verbose=True)

In [ ]:
def decrypt(ciphertext, n_cols=20, verbose=False):
    '''Simplified version'''
    #1. extract head and tail rows
    n_rows = size(ciphertext, n_cols)
    d = n_rows//2 
    s, f = (d, d) if n_rows % 2 == 0 else (d + 1, d)
    head, tail, ciphertext = ciphertext[:s], ciphertext[-f:], ciphertext[s:-f]
    if verbose:
        print('head:', head, 'tail:', tail)
    #2. extracting rows
    chars = []
    for i in range(n_rows):
        row = ciphertext[i::n_rows]
        if i % 2 == 0:
            row = head[i//2] + row
        else:
            row = tail[i//2] + row[::-1]
        if verbose:
            print(f'i: {i:2}', 'row:', row)
        chars += row
    return ''.join(chars)

In [ ]:
text3d = decrypt(ciphertext3, verbose=True)

In [ ]:
text3 in text3d

In [ ]:
def encrypt(text, n_cols=20, verbose=False):
    #1. constructing cols, head and tail
    n_rows = size(text, n_cols)
    head, tail = text[::n_cols*2], text[n_cols::n_cols*2]
    if verbose:
        print('head:', head, 'tail:', tail)
    cols = []
    for i in range(n_rows):
        col = text[i*n_cols:(i+1)*n_cols]
        if i % 2 == 0:
            col = col[1:]
        else:
            col = col[1:][::-1]
        if verbose:
            print(f'i: {i:2}', 'col:', col)
        cols += [col]
    #2. convert cols to rows
    ciphertext = ''
    for i in range(n_cols-1):
        ciphertext += ''.join([col[i] for col in cols])
    #3. add head and tail
    return head + ciphertext + tail

In [ ]:
ciphertext3e = encrypt(text3d, 20, verbose=True)

In [ ]:
ciphertext3e == ciphertext3

In [ ]:
text2

In [ ]:
find_pattern(ciphertext2, ['NLXQAR', 'JYXSHY', 'PYDSD', 'XYJHGW'])

In [ ]:
text2d = decrypt(ciphertext2, 20, verbose=True)

In [ ]:
text2

In [ ]:
text2 in text2d

In [ ]:
text1

In [ ]:
find_pattern(ciphertext1, ['LHTM', 'UKDPR', 'VAPIDK'])

In [ ]:
reshape(ciphertext1, n_rows=20, verbose=True)

In [ ]:
reshape(ciphertext1[28:-27], n_rows=19, verbose=True)

In [ ]:
text1d = decrypt(ciphertext1, verbose=True)

In [ ]:
text1

In [ ]:
text1 in text1d

In [ ]:
ciphertext1e = encrypt(text1d, verbose=True)

In [ ]:
ciphertext1e == ciphertext1

## Ciphertext decryption

In [ ]:
decrypt1(decrypt(ciphertext2))

In [ ]:
df_test['text'] = df_test['ciphertext'].apply(lambda x: decrypt(x)).apply(lambda x: decrypt1(x))

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
df_test['train_index'] = 0

for idx, train_row in tqdm_notebook(df_train.iterrows(), total=df_train.shape[0]):
    l = len(train_row['text'])
    text_trimmed = df_test['text'].apply(lambda x: x[(len(x)-l)//2:(len(x)-l)//2+l])
    cond = text_trimmed == train_row['text']
    if df_test[cond].shape[0] == 1:
        print('.', end='')
        df_test.loc[cond, 'train_index'] = train_row['index']

In [ ]:
df_test.head()

In [ ]:
df_test[df_test['train_index'] == 0]

There is some undetected ciphertexts (maybe some bugs in code :), but major part is detected correctly.

## Results output

In [ ]:
df_test.rename(columns={'train_index':'index'}, inplace=True)

In [ ]:
df_subm = pd.read_csv('../input/ciphertext-challenge-iii/test.csv', usecols=['ciphertext_id'])

In [ ]:
df_subm = df_subm[['ciphertext_id']].merge(df_test[['ciphertext_id', 'index']], how='left',
                                           left_on='ciphertext_id', right_on='ciphertext_id')
df_subm = df_subm.fillna(0)
df_subm['index'] = df_subm['index'].astype(np.int32)

In [ ]:
df_subm.head(10)

In [ ]:
df_subm.to_csv('diff2.csv', index=None)